# Avaliação modelo DTLFE pré-treinado

---


# Configurações

In [1]:
import os
import sys
import gc
import json
from pprint import pprint
from collections import Counter
import copy
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display

from tqdm import *

from pretty_confusion_matrix import *

# TODO: implementar rotina na classe PyNILM.utils
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def listar_variaveis_memoria(ambiente):
    print("* Variáveis instanciadas em memória:")
    print("---")
    total = 0
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in ambiente.items()),
                             key= lambda x: -x[1])[:10]:
        total += size
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    print("---")
    print("Total:", sizeof_fmt(total))
    
# TODO: implementar na classe utils
def highlight_col(x):
    r = 'background-color: #D9D9D9'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, -2] = r
    return df1   

In [2]:
# # CONSTANTES FUNDAMENTAIS DE ORGANIZACAO DE PASTAS/ARQUIVOS
RESIDENCIA = 3

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "28") # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)


In [3]:
from datetime import datetime, timedelta
from dateutil.parser import parse as date_parser

from matplotlib import rcParams
import matplotlib.pyplot as plt
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM
import nilmtk.utils

%matplotlib inline

# Dados

## Base REDD

In [4]:
# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")
caminho_ukdale = os.path.join(caminho_dados, "UK-DALE")

# Path completo do arquivo REDD/UKDALE
arquivo_dataset_redd = os.path.join(caminho_redd, "redd.h5")
arquivo_dataset_ukdale = os.path.join(caminho_ukdale, "ukdale.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

# Gerar arquivo H5 (Nilmtk) do dataset REDD, caso n exista
if not os.path.isfile(arquivo_dataset_redd):
    from nilmtk.dataset_converters import convert_redd
    
    print("Gerando arquivo H5 (NILMTK) da base REDD, aguarde...")
    print("-----")
    convert_redd(caminho_redd, arquivo_dataset_redd)

# Carregando dataset REDD no objeto NILMTK
# Exemplo de carregamento da base REDD no NILMTK
import h5py # * Evitar erro de incompatibilidade entre h5py e nilmtk
from nilmtk import DataSet
from nilmtk.utils import print_dict

redd = DataSet(arquivo_dataset_redd)
print("NILMTK -> Detalhes sobre o dataset REDD:")
print_dict(redd.metadata)
print()

NILMTK -> Detalhes sobre o dataset REDD:


In [14]:
redd.metadata

{'name': 'REDD',
 'long_name': 'The Reference Energy Disaggregation Data set',
 'creators': ['Kolter, Zico', 'Johnson, Matthew'],
 'publication_date': 2011,
 'institution': 'Massachusetts Institute of Technology (MIT)',
 'contact': 'zkolter@cs.cmu.edu',
 'description': 'Several weeks of power data for 6 different homes.',
 'subject': 'Disaggregated power demand from domestic buildings.',
 'number_of_buildings': 6,
 'timezone': 'US/Eastern',
 'geo_location': {'locality': 'Massachusetts',
  'country': 'US',
  'latitude': 42.360091,
  'longitude': -71.09416},
 'related_documents': ['http://redd.csail.mit.edu',
  'J. Zico Kolter and Matthew J. Johnson. REDD: A public data set for energy disaggregation research. In proceedings of the SustKDD workshop on Data Mining Applications in Sustainability, 2011. http://redd.csail.mit.edu/kolter-kddsust11.pdf\n'],
 'schema': 'https://github.com/nilmtk/nilm_metadata/tree/v0.2',
 'meter_devices': {'eMonitor': {'model': 'eMonitor',
   'manufacturer': 'Po

## Base UK-DALE

In [4]:
ukdale = DataSet(arquivo_dataset_ukdale)
print("NILMTK -> Detalhes sobre o dataset UK-DALE:")
print_dict(ukdale.metadata)
print()

NILMTK -> Detalhes sobre o dataset UK-DALE:


**IMPORTANTE:** O fine-tuning do modelo foi realizado utilizando como dados:

1. Base REDD: `30 dias iniciais` de medições de cada aparelho de interesseo;
2. Base UK-DALE: `60 dias iniciais` de registros de cada aparelho de interesse. 

## Melhores Combinações de Taxas e Janelas para cada Aparelho (estudo 19)

In [5]:
df_melhores_taxas_janelas = pd.read_csv(os.path.join(caminho_dados, "19", "melhores_taxa_janela_aparelhos.csv"), index_col=0)
df_melhores_taxas_janelas

,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,dish_washer - 9,2,720,0.05,95.33,20.00,25.00,22.22,59.91
1,fridge - 7,2,1080,0.00,100.00,100.00,100.00,100.00,100.00
2,microwave - 16,2,900,0.04,95.83,66.67,33.33,44.44,71.14
3,washer_dryer - 13,2,60,0.00,99.89,100.00,95.74,97.83,98.88
4,washer_dryer - 14,3,360,0.02,97.99,100.00,55.56,71.43,85.19


In [10]:
# TODO: 
# - Desenvolver módulo da metodologia na lib PyNILM 

## Parâmetros de RP dos Aparelhos (estudo 18)

In [6]:
# Carregando arquivos de parametros, caso n estejam (kernel reiniciado)
if not 'parametros_rp_aparelho' in locals():
    with open(os.path.join(caminho_dados, "18", "parametros_rp_aparelho.json"),'r') as arquivo:
        parametros_rp_aparelho = json.load(arquivo)

## Ambiente e Funções Auxiliares

In [7]:
# from PyNILM.dados.janelas import Janelas
from PyNILM.dados.janelas import Janelas
from PyNILM.dados.utils import *

from PyNILM.avaliacao.metricas import *
from PyNILM.avaliacao.graficos import *
from PyNILM.avaliacao.analises import *

from PyNILM.modelos.utils import *
from PyNILM.modelos.dlafe import DLAFE
from PyNILM.modelos.rqa import RQA

# Inicializar uso GPU
start_tf_session(memory_limit=int(1024*4))

Virtual devices cannot be modified after being initialized


## Configurações do Experimento

In [10]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

# aparelhos = [
#     'dish_washer - 9',
#     'fridge - 7',
#     'microwave - 16',
#     'washer_dryer - 13', 
#     'washer_dryer - 14'
# ]

TAXA = 2 # Fixa

# IMPORTANTE: Agora o modelo extrator será o pre-treinado
# modelo_extrator = transfer_learning.vgg16.VGG16(
#             weights='imagenet', 
#             include_top=False,
#             pooling='avg'
#         )
preprocessamento_extrator = transfer_learning.vgg16.preprocess_input

## Carregando os dados

In [14]:
# https://notebook.community/jaduimstra/nilmtk/docs/manual/user_guide/elecmeter_and_metergroup

In [36]:
atraso_inicial = 30
periodo = 30

residencia = redd.buildings[RESIDENCIA]

# Gerar janelas para cada canal/aparelho
print("* Gerando janelas para cada canal/aparelho...")

# inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
# fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
print('Residencia:', residencia)

for e in residencia.elec.all_meters():

    # Selecionando canal/aparelho
    # e = self.residencia.elec[e_i]

    # Normalizar nome aparelho/canal de medicao
    aparelho = e.label().lower().replace(" ", "_")

    # if not any(map(aparelho.__contains__, aparelhos.keys())) or 'site_meter' in aparelho:
    #     continue

    tamanho_janela = 1080

    # # Extraindo medicoes de energia da carga
    # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
    
    # Extraindo medicoes de energia da carga (toda a serie)
    consumo_aparelho = e.power_series_all_data(sample_period=TAXA).replace(np.nan, 0) # Remover nan (por zero)

    # Definindo periodo/janela de analise (consumo individual e agregado)
    if aparelho not in 'site_meter':
        inicio_periodo = consumo_aparelho.index[0] + timedelta(days=atraso_inicial)
        fim_periodo = inicio_periodo + timedelta(days=atraso_inicial+periodo)
        
        # Selecionando periodo no consumo do aparelho
        indices_aparelho = consumo_aparelho.index.to_pydatetime()
        consumo_aparelho = consumo_aparelho[(indices_aparelho >= inicio_periodo) & (indices_aparelho <= fim_periodo)]
        print(f'Consumo aparelho {aparelho}: de', inicio_periodo, 'a', consumo_aparelho.index[-1],  '|', consumo_aparelho.shape)
    
        break

* Gerando janelas para cada canal/aparelho...
Residencia: Building(instance=3, dataset='REDD')
Consumo aparelho sockets: de 2011-05-16 01:11:30-04:00 a 2011-05-30 20:19:36-04:00 | (639243,)


In [39]:
def carregar_dados(
    base, taxa_amostral=2, 
    atraso_inicial=0, periodo=30, 
    residencias=None, aparelhos=None, 
    estatisticas=False, metadados=False, 
    debug=False):
    """
    Função para carregar e preparar as janelas de consumo e ativações dos aparelhos em diferentes residenciais de um dataset.

    Retorno:
        dados (list): lista com as informações (janelas e metadados) dos aparelhos de interesse.
    """
    dados = []

    for r in base.buildings:
        
        if residencias and r not in residencias:
            continue

        if debug: print(f"> Processando residência #{r}...")
        
        # dados[b] = {
        #     'medidores': [],
        #     'aparelhos': []

        # }

        # Consumo agregado da residencia (serie historica)
        # if debug: print("* Extraindo dados de consumo agregado da residencia...")
        consumo_agregado = carregar_dados_consumo_agregado(
            base, taxa_amostral=taxa_amostral, residencia=r, debug=debug)
        indices_agregado = consumo_agregado.index.to_pydatetime()
        if debug: print(f'  - Consumo agregado: de', consumo_agregado.index[0], 'a', consumo_agregado.index[-1])

        residencia = base.buildings[r]

        # Gerar janelas para cada canal/aparelho
        if debug: print("* Gerando janelas para cada canal/aparelho...")

        # inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
        # fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
        if debug: print('Residencia:', residencia)
        
        for e in residencia.elec.all_meters():

            # Selecionando canal/aparelho
            # e = self.residencia.elec[e_i]

            # Normalizar nome aparelho/canal de medicao
            aparelho = e.label().lower().replace(" ", "_")

            if not any(map(aparelho.__contains__, aparelhos.keys())) or 'site_meter' in aparelho:
                continue

            tamanho_janela = 1080
            if aparelhos:
                for k, v in aparelhos.items():
                    if k in aparelho:
                        tamanho_janela = v
                        break

            try:

                # # Extraindo medicoes de energia da carga
                # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
                
                # Extraindo medicoes de energia da carga (toda a serie)
                consumo_aparelho = e.power_series_all_data(sample_period=taxa_amostral).replace(np.nan, 0) # Remover nan (por zero)

                # Definindo periodo/janela de analise (consumo individual e agregado)
                if aparelho not in 'site_meter':
                    inicio_periodo = consumo_aparelho.index[0] + timedelta(days=atraso_inicial)
                    fim_periodo = inicio_periodo + timedelta(days=atraso_inicial+periodo)
                    
                    # Selecionando periodo no consumo do aparelho
                    indices_aparelho = consumo_aparelho.index.to_pydatetime()
                    consumo_aparelho = consumo_aparelho[(indices_aparelho >= inicio_periodo) & (indices_aparelho <= fim_periodo)]
                    if debug: print(f'Consumo aparelho {aparelho}: de', inicio_periodo, 'a', consumo_aparelho.index[-1], 
                                '|', consumo_aparelho.shape, 
                                '>>>', consumo_agregado.values.min(), consumo_agregado.values.max(),
                                    consumo_agregado.values.mean(), consumo_agregado.values.std())

                    # Selecionando periodo no consumo agregado (mesmo range de analise para as janelas)
                    consumo_agregado_aparelho = consumo_agregado[(indices_agregado >= inicio_periodo) & (indices_agregado <= fim_periodo)]
                    if debug: 
                        print(f'  - Consumo agregado/aparelho: de ', consumo_agregado_aparelho.index[0], 'a', consumo_agregado_aparelho.index[-1],
                                '|', consumo_agregado_aparelho.shape, 
                                '>>>', consumo_agregado_aparelho.values.min(), consumo_agregado_aparelho.values.max(),
                                    consumo_agregado_aparelho.values.mean(), consumo_agregado_aparelho.values.std())
                        print(f'  - Shapes sincronizados?', consumo_agregado_aparelho.shape == consumo_aparelho.shape)

                    # print(' -', aparelho, '=', inicio_periodo, 'a', fim_periodo, '|', tamanho_janela )

                    # TODO: dividir em janelas com indices -> np.array_split(dados, dados.shape[0] // (720 - 1))
                    
                    # Calculando tamanho máximo da série (padding, dependendo tamanho janeka)
                    limite_serie = int(len(consumo_aparelho.values) / tamanho_janela) * tamanho_janela

                    # Garantindo limite da serie valido (caber dentro do reshape do tamanho janela)
                    while limite_serie % tamanho_janela != 0:
                        limite_serie -= 1
                    
                    # Encaixando medicao dentro do tamanho de janelas (p/ fazer reshape)
                    # power = power.values[:limite_serie]
                    consumo_aparelho = consumo_aparelho.iloc[:limite_serie]
                    consumo_agregado_aparelho = consumo_agregado_aparelho.iloc[:limite_serie]

                    # Gerando máscara de status (ativo ou não), considerando ruido da carga
                    # ou rede na medição (threshod)
                    # status = power > e.on_power_threshold()
                    ativacoes = (consumo_aparelho >= e.on_power_threshold()).astype(int)

                    # # Dividindo em janelas (consumo energetico individual e agregado, bem como ativacoes)
                    # windows_series = power.reshape(-1, tamanho_janela)
                    # windows_status = status.reshape(-1, tamanho_janela)
                    # janelas_aparelho = np.array_split(consumo_aparelho, consumo_aparelho.shape[0] // (tamanho_janela - 1))
                    janelas_consumo_aparelho = np.vstack(
                        np.array_split(
                            consumo_aparelho, 
                            consumo_aparelho.shape[0] // tamanho_janela
                        )
                    )
                    janelas_agregado_aparelho = np.vstack(
                        np.array_split(
                            consumo_agregado_aparelho, 
                            consumo_agregado_aparelho.shape[0] // tamanho_janela
                        )
                    )
                    janelas_ativacoes = np.vstack(
                        np.array_split(
                            ativacoes, 
                            ativacoes.shape[0] // tamanho_janela
                            )
                    )

                    # # # Remover nan (por zero)
                    # # windows_series = np.nan_to_num(windows_series)

                    # # Extraindo ativacoes
                    # windows_status = np.where(
                    #     np.sum(windows_status, axis=1) > 0, 1, 0
                    # )  # Estado de cada janela, baseado na pré-avaliação da serie
                    ativacao_por_janela = []
                    for w in janelas_ativacoes:
                        ativacao_por_janela.append(1 if w.sum() > 0 else 0)
                    

                    # #     # Calcular rotulos a partir das janelas
                    # #     # Podendo ser:
                    # #     #   - `estado` (denotando carga ATIVA [1] ou INATIVA [0]);
                    # #     #   - `total`(soma da janela);
                    # #     #   - `media`;
                    # #     rotulos = {
                    # #         "total": np.sum(windows_series, axis=1),
                    # #         "media": np.mean(windows_series, axis=1),
                    # #         "estado": np.where(
                    # #             np.sum(windows_status, axis=1) > 0, 1, 0
                    # #         )  # Estado de cada janela, baseado na pré-avaliação da serie
                    # #         # completa, considerando ruido
                    # #     }

                    d = {
                        "aparelho": aparelho,
                        "instancia": e.instance(),
                        "residencia": r,
                        "janelas": janelas_agregado_aparelho,
                        "consumo": janelas_consumo_aparelho,
                        "status": np.array(ativacao_por_janela)
                        }

                    if estatisticas:
                        d["estatisticas"] = {
                            "status": dict(Counter(ativacao_por_janela)),
                            # TODO: validar integridade das janelas (mesmo tamanho) e conversao das janelas para np.array (np.vstack)
                            "consumo_por_janela": {
                                "min": janelas_agregado_aparelho.sum(axis=1).min(),
                                "max": janelas_agregado_aparelho.sum(axis=1).max(),
                                "mean": janelas_agregado_aparelho.sum(axis=1).mean(),
                                "std": janelas_agregado_aparelho.sum(axis=1).std()
                            },
                            "consumo_historico_aparelho": {
                                "min": consumo_aparelho.min(),
                                "max": consumo_aparelho.max(),
                                "mean": consumo_aparelho.mean(),
                                "std": consumo_aparelho.std()
                            }
                        }
                    
                    if metadados:
                        d["metadata"] = e.metadata

                    dados.append(d)

                    # TODO: Sincronizar medidor e aparelho (indices)

                    #     if self.debug: print(f"{aparelho} -> {windows_series.shape}")

            except Exception as ex:
                if debug: print(f"{aparelho}-{e.instance()}: erro ao extrair dados -> {str(ex)}")
                # return {
                #         "aparelho": aparelho,
                #         "instancia": e.instance(),
                #         "residencia": r,
                #         "janelas": janelas_agregado_aparelho,
                #         "consumo": consumo_aparelho,
                #         "status": np.array(ativacao_por_janela)
                #         }
                
    return dados

def carregar_dados_consumo_agregado(base, taxa_amostral=2, residencia=1, debug=False):
    # Consumo agregado = potencia aparente
    # COnsumo individual = potencia ativa
    dados = []

    for b in base.buildings:
            
        if b != residencia:
            continue

        residencia = base.buildings[b]

        # Gerar janelas para cada canal/aparelho
        if debug: print("* Extraindo consumo agregado dos medidores...")

        # inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
        # fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
        if debug: print('Residencia:', residencia)
        
        for e in residencia.elec.all_meters():

            # Selecionando canal/aparelho
            # e = self.residencia.elec[e_i]

            # Normalizar nome aparelho/canal de medicao
            aparelho = e.label().lower().replace(" ", "_")

            if aparelho != 'site_meter':
                continue

            try:

                # # Extraindo medicoes de energia da carga
                # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
                
                # Extraindo medicoes de energia da carga (toda a serie)
                consumo = e.power_series_all_data(sample_period=taxa_amostral)

                # Remover nan (por zero)
                consumo = consumo.replace(np.nan, 0)

                dados.append({
                    "rotulo": aparelho,
                    "instancia": e.instance(),
                    "consumo": consumo
                })

            except Exception as e:
                if debug: print(f"{aparelho}: erro ao extrair dados -> {str(e)}")

    consumo_agregado = pd.concat([c['consumo'] for c in dados], axis=1).sum(1, min_count=1).fillna(0)

    return consumo_agregado

In [40]:
def obter_tipo_aparelho(
    aparelho, 
    tipos = {
        'dish_washer': 'dish washer',
        'fridge': 'fridge',
        'fridge_freezer': 'fridge',
        'microwave': 'microwave',
        'washer_dryer': 'washer_dryer',
        'washing_machine': 'washing_machine'
    }):
    if not any(map(aparelho.__contains__, aparelhos.keys())) or 'site_meter' in aparelho:
        return None
    else:
        return tipos[aparelho]
        

In [41]:
from pathlib import Path
from glob import glob

from PyNILM.dados.utils import *
from PyNILM.modelos.utils import *
from pyts.image import RecurrencePlot

def converter_serie_para_rp(
    serie,
    input_shape=TAMANHO_IMAGEM_DLAFE,
    data_type=np.float32,
    normalize=False, 
    standardize=False, 
    rescale=False,
    # persistir=True,
    # deletar_arquivo_amostra=True
    ):
    """Função de pré-processamento tf.data"""
    # Carregando janelas de consumo (X) e estados da carga (y)
    
    # import os
    # import numpy as np
    # from pathlib import Path
    # from pyts.image import RecurrencePlot
    
    # X = np.load(amostra).astype(data_type)
    # y = np.int8(Path(str(amostra)).stem.split('-')[-1])

    # Transformando janela de consumo em imagem RP
    img = RecurrencePlot(**PARAMETROS_RP).fit_transform([serie])[0]
    img = cv2.resize(
            img, 
            dsize=input_shape[:2], 
            interpolation=cv2.INTER_CUBIC
        ).astype(data_type)

    if np.sum(img) > 0:
        # TODO: improve fit/predict statistics
        # Normalizar
        if normalize:
            img = (img - img.min()) / (img.max() - img.min()) # MinMax (0,1)
            #img = (img - img.mean()) / np.max([img.std(), 1e-4])

        # Padronizar
        elif standardize:
            img = (img - img.mean())/img.std()#tf.image.per_image_standardization(img).numpy()
            
        elif rescale:
            img = (img - img.min()) / (img.max() - img.min())

    # N canais
    X_rp = np.stack([img for _ in range(input_shape[-1])],axis=-1).astype(data_type)  

    return X_rp

In [42]:
# Periodo em dias para o estudo
PERIODO = 60

# Consolidando dados ukdale (pré-treino)
dados_ukdale = carregar_dados(
    ukdale,  
    atraso_inicial=PERIODO, # Contemplar dados nao vistos durante fine-tuning
    periodo=PERIODO,
    residencias=None,
    aparelhos={
        'microwave': 900, 
        'dish': 720, 
        'washer_dryer': 360, 
        'washing_machine': 360, 
        'fridge': 1080
        },
    estatisticas=False,
    metadados=False,
    debug=True
    )

> Processando residência #1...
* Extraindo consumo agregado dos medidores...
Residencia: Building(instance=1, dataset='UK-DALE')
  - Consumo agregado: de 2012-11-09 22:28:14+00:00 a 2015-01-05 06:27:12+00:00
* Gerando janelas para cada canal/aparelho...
Residencia: Building(instance=1, dataset='UK-DALE')
Consumo aparelho washer_dryer: de 2013-01-08 22:28:18+00:00 a 2013-05-08 23:28:18+01:00 | (5184001,) >>> 0.0 16442.965 613.57794 826.2863
  - Consumo agregado/aparelho: de  2013-01-08 22:28:18+00:00 a 2013-05-08 23:28:18+01:00 | (5184001,) >>> 0.0 12559.26 461.54785 708.40137
  - Shapes sincronizados? True
Consumo aparelho dish_washer: de 2013-01-08 22:28:18+00:00 a 2013-05-08 23:28:18+01:00 | (5184001,) >>> 0.0 16442.965 613.57794 826.2863
  - Consumo agregado/aparelho: de  2013-01-08 22:28:18+00:00 a 2013-05-08 23:28:18+01:00 | (5184001,) >>> 0.0 12559.26 461.54785 708.40137
  - Shapes sincronizados? True
Consumo aparelho fridge_freezer: de 2013-02-12 22:21:32+00:00 a 2013-06-12 23:2

In [44]:
# Exportando base de teste (redd)
dados_redd = carregar_dados(
    redd,  
    atraso_inicial=30,
    periodo=30,
    residencias=None,
    aparelhos={
        'microwave': 900, 
        'dish': 720, 
        'washer dryer': 360, 
        'washing machine': 360, 
        'fridge': 1080
        },
    estatisticas=False,
    metadados=False,
    debug=True
    )

> Processando residência #1...
* Extraindo consumo agregado dos medidores...
Residencia: Building(instance=1, dataset='REDD')
  - Consumo agregado: de 2011-04-18 09:22:08-04:00 a 2011-05-24 15:57:02-04:00
* Gerando janelas para cada canal/aparelho...
Residencia: Building(instance=1, dataset='REDD')
Consumo aparelho fridge: de 2011-05-18 09:22:12-04:00 a 2011-05-24 15:56:34-04:00 | (271032,) >>> 0.0 11870.33 192.70941 584.5038
  - Consumo agregado/aparelho: de  2011-05-18 09:22:12-04:00 a 2011-05-24 15:57:02-04:00 | (271046,) >>> 0.0 7344.645 173.13284 643.75323
  - Shapes sincronizados? False
Consumo aparelho dish_washer: de 2011-05-18 09:22:12-04:00 a 2011-05-24 15:56:34-04:00 | (271032,) >>> 0.0 11870.33 192.70941 584.5038
  - Consumo agregado/aparelho: de  2011-05-18 09:22:12-04:00 a 2011-05-24 15:57:02-04:00 | (271046,) >>> 0.0 7344.645 173.13284 643.75323
  - Shapes sincronizados? False
Consumo aparelho microwave: de 2011-05-18 09:22:12-04:00 a 2011-05-24 15:56:34-04:00 | (271032,

In [ ]:
# ## Apagando registros
# if dados_ukdale: del dados_ukdale
# if dados_redd: del dados_redd
# gc.collect()

# Design Experimental
---

Pré-treinar os modelos com base nos dados de UK-DALE e testar com dados REDD.

In [47]:
for d in dados_ukdale:
    print(d['aparelho'], d['instancia'], d['janelas'].shape)

washer_dryer 5 (14400, 360)
dish_washer 6 (7200, 720)
fridge_freezer 12 (4800, 1080)
microwave 13 (5760, 900)
washing_machine 12 (9878, 360)
dish_washer 13 (4939, 720)
fridge 14 (3292, 1080)
microwave 15 (3951, 900)
washing_machine 6 (14400, 360)
fridge_freezer 19 (3082, 1080)
dish_washer 22 (4624, 720)
microwave 23 (3699, 900)
washer_dryer 24 (9247, 360)


In [48]:
for d in dados_redd:
    print(d['aparelho'], d['instancia'], d['janelas'].shape)

fridge 5 (250, 1080)
dish_washer 6 (376, 720)
microwave 11 (301, 900)
microwave 6 (228, 900)
fridge 9 (190, 1080)
dish_washer 10 (286, 720)
fridge 7 (591, 1080)
dish_washer 9 (887, 720)
microwave 16 (710, 900)
dish_washer 15 (1078, 720)
microwave 3 (663, 900)
fridge 18 (553, 1080)
dish_washer 20 (829, 720)


# Metodologia DLAFE

In [18]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

# Teste da classe
janelas_treino = Janelas(
    base=DataSet(arquivo_dataset_redd),
    id_residencia=3,
    inicio_intervalo='2011-04-16 00:00:00',
    fim_intervalo='2011-05-16 23:59:59',
    debug = False
)

janelas_teste = Janelas(
    base=DataSet(arquivo_dataset_redd),
    id_residencia=3,
    inicio_intervalo='2011-05-17 00:00:00',
    fim_intervalo='2011-05-30 23:59:59',
    debug = False
)

aparelhos = [
    'dish_washer - 9',
    'fridge - 7',
    'microwave - 16',
    'washer_dryer - 13', 
    'washer_dryer - 14'
]

TAXA = 2 # Fixa

modelo_extrator = transfer_learning.vgg16.VGG16(
            weights='imagenet', 
            include_top=False,
            pooling='avg'
        )
preprocessamento_extrator = transfer_learning.vgg16.preprocess_input

def carregar_modelo_pretreinado(
    aparelho,
    caminho_modelos_salvos=r'H:\Meu Drive\phd-thesis\datasets\transfer-learning\periodo-60\modelos_salvos',
    debug=False):
    from glob import glob

    caminho_modelo = glob(os.path.join(caminho_modelos_salvos, f'*{aparelho[:4]}*-final*'))[0]
    if debug: print('Modelo persistido no arquivo:', caminho_modelo)

    modelo = tf.keras.models.load_model(caminho_modelo)
    # return modelo
    return tf.keras.models.Model(
        modelo.input, 
        modelo.layers[-2].output
        )

    # Removendo camadas adicionais (extra-vgg16), incluidas no fine-tuning
    

### Teste de carregamento de modelo pré-treinado

In [94]:
for rotulo_aparelho in aparelhos:
    
    print(rotulo_aparelho)#, glob(os.path.join(caminho_modelos_pretreinados, f'*{rotulo_aparelho[:4]}*-final*')) )
    modelo = carregar_modelo_pretreinado(rotulo_aparelho)
    break

dish_washer - 9


In [95]:
modelo.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d_2   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


## SVM

In [46]:
modelo = SVC(kernel='rbf', random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in aparelhos:
    
    print(f"* Aparelho `{rotulo_aparelho}`...\n")

    # MOdelo extrator pré-treinado para aparelho
    modelo_extrator = carregar_modelo_pretreinado(rotulo_aparelho)
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAMANHO_JANELA = config_aparelho["janela"]

    #######################################################################
    #                AVALIACAO 1 - Base de treino / CV                    #
    #######################################################################
    # Extrair series divididas em janelas para cada medidor
    print("   - Base de TREINO\n")
    print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
        TAXA, TAMANHO_JANELA
    ))
    X, y = carregar_dados_aparelho(
        janelas=janelas_treino,
        instancia=INSTANCIA,
        aparelho=CARGA,
        tamanho_janela=TAMANHO_JANELA,
        taxa=TAXA,
        eliminar_janelas_vazias=True
    )
    print("     -> Detalhes da amostragem (lotes):")
    print("     ---")
    for item in Counter(y).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
    print()
    
    y_true, y_pred  = [], []

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

        # Preparando lotes
        X_treino, X_teste = X[idx_treino], X[idx_teste]
        y_treino, y_teste = y[idx_treino], y[idx_teste]

        # Treinando modelo
        dlafe = DLAFE(
            feature_extractor=modelo_extrator,
            preprocess_input=preprocessamento_extrator,
            classifier=clone(modelo),
            rp_params = PARAMETROS_RP,
            input_shape = modelo_extrator.input_shape[1:], # TAMANHO_IMAGEM_DLAFE,
            normalize=False
        )
        dlafe.fit(X_treino, y_treino)

        # Prevendo conjunto de teste
        y_hat = dlafe.predict(X_teste)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("treino")
        
        reset_tf_session(model_name='dlafe')

        # Extendendo rotulos (analise global)
        y_true.extend(y_teste)
        y_pred.extend(y_hat)
        
    #######################################################################
    #                 AVALIACAO 2 - Base de teste / CV                    #
    #######################################################################
    print("   - Base de TESTE\n")
    print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
        TAXA, TAMANHO_JANELA
    ))

    # Avaliar na base de teste
    X_teste, y_teste = carregar_dados_aparelho(
        janelas=janelas_teste,
        instancia=INSTANCIA,
        aparelho=CARGA,
        tamanho_janela=TAMANHO_JANELA,
        taxa=TAXA,
        eliminar_janelas_vazias=True
    )
    
    print("     -> Detalhes da amostragem (lotes):")
    print("     ---")
    for item in Counter(y_teste).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_teste)*100,1)}%)" )
    print()

    # Treinando modelo
    dlafe = DLAFE(
        feature_extractor=modelo_extrator,
        preprocess_input=preprocessamento_extrator,
        classifier=clone(modelo),
        rp_params = PARAMETROS_RP,
        input_shape = TAMANHO_IMAGEM_DLAFE,
        normalize=False
    )
    dlafe.fit(X, y)

    # Prevendo conjunto de teste
    y_hat = dlafe.predict(X_teste)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append(it+1)
    resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("teste")
    
    reset_tf_session(model_name='dlafe')
    
    print()
    print("   - Final Results:")
    print("   ---")
    print()
    
    print("***** TRAIN *****")
    print("      -> Classification Report:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Confusion Matrix:")
    print()
    print(confusion_matrix(y_true, y_pred))
    print()
    
    print("***** TEST *****")
    print("      -> Classification Report:")
    print()
    print(classification_report(y_teste, y_hat))
    print("      -> Confusion Matrix:")
    print()
    print(confusion_matrix(y_teste, y_hat))
    print()
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_svm.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

* Aparelho `dish_washer - 9`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=720)...
Loading data for meter ElecMeterID(instance=14, building=3, dataset='REDD')     
Done loading data all meters for this chunk.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 623 amostras (97.5%)
        - Classe `1`: 16 amostras (2.5%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

: 

: 

## XGBOOST

In [12]:
def pos_weight(y):
    try:
        counter = Counter(y)
        return counter[0]/counter[1]
    except:
        return 1

In [13]:
modelo = XGBClassifier(random_state=SEED, n_jobs=4)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in aparelhos:
    
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAMANHO_JANELA = config_aparelho["janela"]

    #######################################################################
    #                AVALIACAO 1 - Base de treino / CV                    #
    #######################################################################
    # Extrair series divididas em janelas para cada medidor
    print("   - Base de TREINO\n")
    print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
        TAXA, TAMANHO_JANELA
    ))
    X, y = carregar_dados_aparelho(
        janelas=janelas_treino,
        instancia=INSTANCIA,
        aparelho=CARGA,
        tamanho_janela=TAMANHO_JANELA,
        taxa=TAXA,
        eliminar_janelas_vazias=True
    )
    print("     -> Detalhes da amostragem (lotes):")
    print("     ---")
    for item in Counter(y).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
    print()
    
    y_true, y_pred  = [], []

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

        # Preparando lotes
        X_treino, X_teste = X[idx_treino], X[idx_teste]
        y_treino, y_teste = y[idx_treino], y[idx_teste]

        # Treinando modelo
        dlafe = DLAFE(
            feature_extractor=modelo_extrator,
            preprocess_input=preprocessamento_extrator,
            classifier=clone(modelo),#.set_params(**{'scale_pos_weight': pos_weight(y)}),
            rp_params = PARAMETROS_RP,
            input_shape = modelo_extrator.input_shape[1:], #TAMANHO_IMAGEM_DLAFE,
            normalize=False
        )
        dlafe.fit(X_treino, y_treino)

        # Prevendo conjunto de teste
        y_hat = dlafe.predict(X_teste)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("treino")
        
        reset_tf_session(model_name='dlafe')

        # Extendendo rotulos (analise global)
        y_true.extend(y_teste)
        y_pred.extend(y_hat)
        
    #######################################################################
    #                 AVALIACAO 2 - Base de teste / CV                    #
    #######################################################################
    print("   - Base de TESTE\n")
    print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
        TAXA, TAMANHO_JANELA
    ))

    # Avaliar na base de teste
    X_teste, y_teste = carregar_dados_aparelho(
        janelas=janelas_teste,
        instancia=INSTANCIA,
        aparelho=CARGA,
        tamanho_janela=TAMANHO_JANELA,
        taxa=TAXA,
        eliminar_janelas_vazias=True
    )
    
    print("     -> Detalhes da amostragem (lotes):")
    print("     ---")
    for item in Counter(y_teste).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_teste)*100,1)}%)" )
    print()

    # Treinando modelo
    dlafe = DLAFE(
        feature_extractor=modelo_extrator,
        preprocess_input=preprocessamento_extrator,
        classifier=clone(modelo),#.set_params(**{'scale_pos_weight': pos_weight(y)}),
        rp_params = PARAMETROS_RP,
        input_shape = TAMANHO_IMAGEM_DLAFE,
        normalize=False
    )
    dlafe.fit(X, y)

    # Prevendo conjunto de teste
    y_hat = dlafe.predict(X_teste)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append(it+1)
    resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("teste")
    
    reset_tf_session(model_name='dlafe')
    
    print()
    print("   - Final Results:")
    print("   ---")
    print()
    
    print("***** TRAIN *****")
    print("      -> Classification Report:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Confusion Matrix:")
    print()
    print(confusion_matrix(y_true, y_pred))
    print()
    
    print("***** TEST *****")
    print("      -> Classification Report:")
    print()
    print(classification_report(y_teste, y_hat))
    print("      -> Confusion Matrix:")
    print()
    print(confusion_matrix(y_teste, y_hat))
    print()
    
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_xgboost.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

* Aparelho `dish_washer - 9`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=720)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 623 amostras (97.5%)
        - Classe `1`: 16 amostras (2.5%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

[12:52:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[12:52:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[12:52:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

  0%|          | 0/10 [00:00<?, ?it/s]

[12:54:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[12:54:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[12:54:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

...

  0%|          | 0/10 [00:00<?, ?it/s]

[12:58:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[12:59:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[13:00:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

  0%|          | 0/10 [00:00<?, ?it/s]

[13:10:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[13:10:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[13:11:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

acc                                      f1  \
                              mean       std       max       min      mean   
appliance         base                                                       
dish_washer - 9   teste   0.968137       NaN  0.968137  0.968137  0.491905   
                  treino  0.973413  0.007508  0.984375  0.968750  0.493257   
fridge - 7        teste   0.981752       NaN  0.981752  0.981752  0.495396   
                  treino  0.995349  0.009806  1.000000  0.976744  0.898824   
microwave - 16    teste   0.960123       NaN  0.960123  0.960123  0.803751   
                  treino  0.955015  0.022662  1.000000  0.921569  0.701603   
washer_dryer - 13 teste   0.997725       NaN  0.997725  0.997725  0.984742   
                  treino  0.998408  0.000839  1.000000  0.997347  0.982962   
washer_dryer - 14 teste   0.987654       NaN  0.987654  0.987654  0.938601   
                  treino  0.987402  0.007607  1.000000  0.976378  0.905937   

                                                             auc            \
                               std       max       min      mean       std   
appliance         base                                                       
dish_washer - 9   teste        NaN  0.491905  0.491905  0.500000       NaN   
                  treino  0.001922  0.496063  0.492063  0.499206  0.002510   
fridge - 7        teste        NaN  0.495396  0.495396  0.500000       NaN   
                  treino  0.213299  1.000000  0.494118  0.500000  0.000000   
microwave - 16    teste        NaN  0.803751  0.803751  0.768464       NaN   
                  treino  0.178837  1.000000  0.479592  0.678189  0.167578   
washer_dryer - 13 teste        NaN  0.984742  0.984742  0.973851       NaN   
                  treino  0.009378  1.000000  0.969910  0.972817  0.018423   
washer_dryer - 14 teste        NaN  0.938601  0.938601  0.895833       NaN   
                  treino  0.066179  1.000000  0.779641  0.861257  0.089757   

                                              
                               max       min  
appliance         base                        
dish_washer - 9   teste   0.500000  0.500000  
                  treino  0.500000  0.492063  
fridge - 7        teste   0.500000  0.500000  
                  treino  0.500000  0.500000  
microwave - 16    teste   0.768464  0.768464  
                  treino  1.000000  0.479592  
washer_dryer - 13 teste   0.973851  0.973851  
                  treino  1.000000  0.944444  
washer_dryer - 14 teste   0.895833  0.895833  
                  treino  1.000000  0.700000

## MLP

In [14]:
def class_weight(y, debug=False):
    
    # Classes distribution
    neg, pos = np.bincount(y)
    total = neg + pos

    # Scaling by total/2 helps keep the loss to a similar magnitude.
    # The sum of the weights of all examples stays the same.
    w_0 = (1 / neg)*(total)/2.0 
    w_1 = (1 / pos)*(total)/2.0

    class_weight = {0: w_0, 1: w_1}

    print('Weight for class 0: {:.2f}'.format(w_0))
    print('Weight for class 1: {:.2f}'.format(w_1))
    
    return class_weight

In [15]:
def mlp(
    input_shape=TAMANHO_IMAGEM_DLAFE,
    metrics=[
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ], 
    output_bias=None
):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    model = keras.Sequential([
      keras.layers.Dense(10, activation='relu', input_shape=input_shape),
      # keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias),
    ])

    model.compile(
      optimizer=keras.optimizers.Adam(lr=0.001),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics
    )

    return model

In [16]:
modelo = MLPClassifier(alpha=1e-3, hidden_layer_sizes=(10,), random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in aparelhos:
    
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAMANHO_JANELA = config_aparelho["janela"]

    #######################################################################
    #                AVALIACAO 1 - Base de treino / CV                    #
    #######################################################################
    # Extrair series divididas em janelas para cada medidor
    print("   - Base de TREINO\n")
    print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
        TAXA, TAMANHO_JANELA
    ))
    X, y = carregar_dados_aparelho(
        janelas=janelas_treino,
        instancia=INSTANCIA,
        aparelho=CARGA,
        tamanho_janela=TAMANHO_JANELA,
        taxa=TAXA,
        eliminar_janelas_vazias=True
    )
    print("     -> Detalhes da amostragem (lotes):")
    print("     ---")
    for item in Counter(y).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
    print()
    
    y_true, y_pred  = [], []

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

        # Preparando lotes
        X_treino, X_teste = X[idx_treino], X[idx_teste]
        y_treino, y_teste = y[idx_treino], y[idx_teste]

        # Treinando modelo
        dlafe = DLAFE(
            feature_extractor=modelo_extrator,
            preprocess_input=preprocessamento_extrator,
            classifier=clone(modelo),
            rp_params = PARAMETROS_RP,
            input_shape = TAMANHO_IMAGEM_DLAFE,
            normalize=False
        )
        dlafe.fit(X_treino, y_treino)

        # Prevendo conjunto de teste
        y_hat = dlafe.predict(X_teste)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("treino")
        
        reset_tf_session(model_name='dlafe')

        # Extendendo rotulos (analise global)
        y_true.extend(y_teste)
        y_pred.extend(y_hat)
        
    #######################################################################
    #                 AVALIACAO 2 - Base de teste / CV                    #
    #######################################################################
    print("   - Base de TESTE\n")
    print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
        TAXA, TAMANHO_JANELA
    ))

    # Avaliar na base de teste
    X_teste, y_teste = carregar_dados_aparelho(
        janelas=janelas_teste,
        instancia=INSTANCIA,
        aparelho=CARGA,
        tamanho_janela=TAMANHO_JANELA,
        taxa=TAXA,
        eliminar_janelas_vazias=True
    )
    
    print("     -> Detalhes da amostragem (lotes):")
    print("     ---")
    for item in Counter(y_teste).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
    print()

    # Treinando modelo
    dlafe = DLAFE(
        feature_extractor=modelo_extrator,
        preprocess_input=preprocessamento_extrator,
        classifier=clone(modelo),
        rp_params = PARAMETROS_RP,
        input_shape = TAMANHO_IMAGEM_DLAFE,
        normalize=False
    )
    dlafe.fit(X, y)

    # Prevendo conjunto de teste
    y_hat = dlafe.predict(X_teste)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append(it+1)
    resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("teste")
    
    reset_tf_session(model_name='dlafe')
    
    print()
    print("   - Final Results:")
    print("   ---")
    print()
    
    print("***** TRAIN *****")
    print("      -> Classification Report:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Confusion Matrix:")
    print()
    print(confusion_matrix(y_true, y_pred))
    print()
    
    print("***** TEST *****")
    print("      -> Classification Report:")
    print()
    print(classification_report(y_teste, y_hat))
    print("      -> Confusion Matrix:")
    print()
    print(confusion_matrix(y_teste, y_hat))
    print()
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_mlp.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

* Aparelho `dish_washer - 9`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=720)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 623 amostras (97.5%)
        - Classe `1`: 16 amostras (2.5%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=720)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 395 amostras (61.8%)
       - Classe `1`: 13 amostras (2.0%)

Virtual devices cannot be modified after being initialized

   - Final R

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=1080)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `1`: 269 amostras (63.0%)
       - Classe `0`: 5 amostras (1.2%)

Virtual devices cannot be modified after being initialized

   - Final R

...

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=60)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 4643 amostras (61.6%)
       - Classe `1`: 192 amostras (2.5%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=360)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 762 amostras (60.0%)
       - Classe `1`: 48 amostras (3.8%)

Virtual devices cannot be modified after being initialized

   - Final R

acc                                      f1  \
                              mean       std       max       min      mean   
appliance         base                                                       
dish_washer - 9   teste   0.968137       NaN  0.968137  0.968137  0.558552   
                  treino  0.967163  0.015508  0.984375  0.937500  0.524952   
fridge - 7        teste   0.981752       NaN  0.981752  0.981752  0.638236   
                  treino  0.997674  0.007354  1.000000  0.976744  0.949412   
microwave - 16    teste   0.963190       NaN  0.963190  0.963190  0.813821   
                  treino  0.945173  0.022321  0.980392  0.901961  0.696472   
washer_dryer - 13 teste   0.991934       NaN  0.991934  0.991934  0.946178   
                  treino  0.995093  0.002259  0.997347  0.992042  0.948597   
washer_dryer - 14 teste   0.969136       NaN  0.969136  0.969136  0.829561   
                  treino  0.970866  0.011166  0.992126  0.952756  0.774468   

                                                             auc            \
                               std       max       min      mean       std   
appliance         base                                                       
dish_washer - 9   teste        NaN  0.558552  0.558552  0.537196       NaN   
                  treino  0.107013  0.829333  0.483871  0.520200  0.081007   
fridge - 7        teste        NaN  0.638236  0.638236  0.598141       NaN   
                  treino  0.159974  1.000000  0.494118  0.550000  0.158114   
microwave - 16    teste        NaN  0.813821  0.813821  0.770098       NaN   
                  treino  0.137656  0.894845  0.484848  0.695918  0.129340   
washer_dryer - 13 teste        NaN  0.946178  0.946178  0.938381       NaN   
                  treino  0.023468  0.973004  0.914629  0.942625  0.029441   
washer_dryer - 14 teste        NaN  0.829561  0.829561  0.768865       NaN   
                  treino  0.101951  0.942404  0.632736  0.739631  0.116070   

                                              
                               max       min  
appliance         base                        
dish_washer - 9   teste   0.537196  0.537196  
                  treino  0.750000  0.483871  
fridge - 7        teste   0.598141  0.598141  
                  treino  1.000000  0.500000  
microwave - 16    teste   0.770098  0.770098  
                  treino  0.833333  0.489796  
washer_dryer - 13 teste   0.938381  0.938381  
                  treino  0.973004  0.893376  
washer_dryer - 14 teste   0.768865  0.768865  
                  treino  0.900000  0.595902

# Análise dos Resultados (1)

In [17]:
df_resultados_svm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_svm.xlsx"), engine="openpyxl")
df_resultados_svm["model"] = "SVM"

df_resultados_xgboost = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_xgboost.xlsx"), engine="openpyxl")
df_resultados_xgboost["model"] = "XGBOOST"

df_resultados_mlp = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_mlp.xlsx"), engine="openpyxl")
df_resultados_mlp["model"] = "MLP"

# df_resultados_elm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_elm.xlsx"), engine="openpyxl)
# df_resultados_elm["model"] = "ELM"

df_analise = pd.concat([
    df_resultados_svm,
    df_resultados_xgboost,
    df_resultados_mlp, 
#     df_resultados_elm,  
])

print("* Análise por modelo:")
df_analise_modelo = df_analise.groupby(["model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("model")
display(df_analise_modelo)
df_analise_modelo.to_excel(os.path.join(caminho_dados_notebook, "df_analise_modelo.xlsx"))

print()
print("* Análise por aparelho/modelo:")
df_analise_aparelho = df_analise.groupby(["appliance","model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "df_analise_aparelho.xlsx"))

* Análise por modelo:


base       acc                                      f1            \
                     mean       std       max       min      mean       std   
model                                                                         
XGBOOST  treino  0.981917  0.019850  1.000000  0.921569  0.796517  0.217562   
MLP      treino  0.975194  0.023544  1.000000  0.901961  0.778780  0.196613   
SVM      treino  0.980913  0.019488  1.000000  0.941176  0.763340  0.231737   
MLP       teste  0.974830  0.011760  0.991934  0.963190  0.757269  0.156339   
XGBOOST   teste  0.979078  0.015062  0.997725  0.960123  0.742879  0.237037   
SVM       teste  0.977727  0.016924  0.997104  0.953988  0.719217  0.234460   

                                  auc                                
              max       min      mean       std       max       min  
model                                                                
XGBOOST  1.000000  0.479592  0.702294  0.209000  1.000000  0.479592  
MLP      1.000000  0.483871  0.689675  0.186874  1.000000  0.483871  
SVM      1.000000  0.484848  0.675331  0.210834  1.000000  0.500000  
MLP      0.946178  0.558552  0.722536  0.158763  0.938381  0.537196  
XGBOOST  0.984742  0.491905  0.727630  0.220349  0.973851  0.500000  
SVM      0.980825  0.491905  0.701324  0.225873  0.976024  0.500000


* Análise por aparelho/modelo:


acc                                \
                                      mean       std       max       min   
appliance         model   base                                             
dish_washer - 9   MLP     teste   0.968137       NaN  0.968137  0.968137   
                          treino  0.967163  0.015508  0.984375  0.937500   
                  SVM     teste   0.968137       NaN  0.968137  0.968137   
                          treino  0.974975  0.008037  0.984375  0.968750   
                  XGBOOST teste   0.968137       NaN  0.968137  0.968137   
                          treino  0.973413  0.007508  0.984375  0.968750   
fridge - 7        MLP     teste   0.981752       NaN  0.981752  0.981752   
                          treino  0.997674  0.007354  1.000000  0.976744   
                  SVM     teste   0.981752       NaN  0.981752  0.981752   
                          treino  0.995349  0.009806  1.000000  0.976744   
                  XGBOOST teste   0.981752       NaN  0.981752  0.981752   
                          treino  0.995349  0.009806  1.000000  0.976744   
microwave - 16    MLP     teste   0.963190       NaN  0.963190  0.963190   
                          treino  0.945173  0.022321  0.980392  0.901961   
                  SVM     teste   0.953988       NaN  0.953988  0.953988   
                          treino  0.949095  0.010225  0.961538  0.941176   
                  XGBOOST teste   0.960123       NaN  0.960123  0.960123   
                          treino  0.955015  0.022662  1.000000  0.921569   
washer_dryer - 13 MLP     teste   0.991934       NaN  0.991934  0.991934   
                          treino  0.995093  0.002259  0.997347  0.992042   
                  SVM     teste   0.997104       NaN  0.997104  0.997104   
                          treino  0.997745  0.001092  1.000000  0.996021   
                  XGBOOST teste   0.997725       NaN  0.997725  0.997725   
                          treino  0.998408  0.000839  1.000000  0.997347   
washer_dryer - 14 MLP     teste   0.969136       NaN  0.969136  0.969136   
                          treino  0.970866  0.011166  0.992126  0.952756   
                  SVM     teste   0.987654       NaN  0.987654  0.987654   
                          treino  0.987402  0.006640  1.000000  0.976378   
                  XGBOOST teste   0.987654       NaN  0.987654  0.987654   
                          treino  0.987402  0.007607  1.000000  0.976378   

                                        f1                                \
                                      mean       std       max       min   
appliance         model   base                                             
dish_washer - 9   MLP     teste   0.558552       NaN  0.558552  0.558552   
                          treino  0.524952  0.107013  0.829333  0.483871   
                  SVM     teste   0.491905       NaN  0.491905  0.491905   
                          treino  0.493657  0.002057  0.496063  0.492063   
                  XGBOOST teste   0.491905       NaN  0.491905  0.491905   
                          treino  0.493257  0.001922  0.496063  0.492063   
fridge - 7        MLP     teste   0.638236       NaN  0.638236  0.638236   
                          treino  0.949412  0.159974  1.000000  0.494118   
                  SVM     teste   0.495396       NaN  0.495396  0.495396   
                          treino  0.898824  0.213299  1.000000  0.494118   
                  XGBOOST teste   0.495396       NaN  0.495396  0.495396   
                          treino  0.898824  0.213299  1.000000  0.494118   
microwave - 16    MLP     teste   0.813821       NaN  0.813821  0.813821   
                          treino  0.696472  0.137656  0.894845  0.484848   
                  SVM     teste   0.688038       NaN  0.688038  0.688038   
                          treino  0.536889  0.107016  0.740000  0.484848   
                  XGBOOST teste   0.803751       NaN  0.803751  0.803751   
          

# Conclusões

...

# Fim.

In [18]:
%load_ext watermark

In [19]:
%watermark -a "Diego Luiz Cavalca" -u -n -t -z -v -m -g

Author: Diego Luiz Cavalca

Last updated: Sun May 02 2021 13:36:14Hora oficial do Brasil

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Git hash: a29eb3e98689f89f3597358428a2cab6bb3ab9b0

